# SETUP

## is GPU on?

In [1]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

## boilerplate

In [3]:
from tensorflow.keras.callbacks import Callback, EarlyStopping
from numpy import arange

MODEL_NAME = 'resnet50'
IMG_SIZE = (224, 224)
INPUT_SHAPE=(224, 224, 3)
CLASSES = 2
TRIAL = TRIAL = list(arange(3,10)) #list(arange(0,3))
FT_BLOCK = list(arange(0,11)) # FROM feature extractor TO fine tuning scratch
BATCH_SIZE = [32, 64, 128, 256] # these data points will be passed as a batch at one time to the network

## UDC/Fs

In [4]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import layers
from math import floor
from timeit import default_timer as timer
from json import dump
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import data

def model_constructor(FT_BLOCK):
    base_model = ResNet50(
        weights='imagenet', 
        include_top=False,
        input_shape=INPUT_SHAPE)
    
    # construct the top layer containing 
    x = base_model.output
    x = layers.GlobalAveragePooling2D(name='avg_pool')(x) # add a global spatial average pooling layer
    x = layers.Dense(CLASSES, activation='sigmoid',  name='predictions')(x) # and add the output layer for binary class

    # model is ready to be trained
    model = Model(inputs=base_model.input, outputs=x)

    # freeze the layers before the `FROZEN_END` layer 
    total_layers = len(base_model.layers)
    ft_layers = floor(total_layers / 10)
    FROZEN_END = total_layers - ft_layers * FT_BLOCK

    base_model.trainable = True
    for layer in base_model.layers[:FROZEN_END]: 
        layer.trainable = False
    
    return model

class TimeCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

def time_converter(sec):
    hours, rem = divmod(sec, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

def save_history(history, tag):
    file_path = f'{HISTORY_DIR}/{tag}.json'
    with open(file_path, 'w') as f:
        dump(history.history, f)
        
def data_preparation(BATCH_SIZE):
    train_dir = '../../data/binaryclass_clean/train/'
    test_dir = '../../data/binaryclass_clean/test/'

    train_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='training')

    val_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='validation')

    test_ds = image_dataset_from_directory(
        directory=test_dir,
        label_mode='categorical',
        batch_size=1,
        image_size=IMG_SIZE)

    AUTOTUNE = data.AUTOTUNE
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

# GRID SEARCH

In [4]:
for i in TRIAL:
    print(" - - - - - TRIAL:", i, " - - - - - ")
    HISTORY_DIR = f'../../logs/FT/{i}'
    for batch in BATCH_SIZE:
        print("Batch size:", batch)
        train_ds, val_ds, test_ds = data_preparation(batch)
        epochs, loss, accuracy = [None]*len(FT_BLOCK), [None]*len(FT_BLOCK), [None]*len(FT_BLOCK)
        for ft in FT_BLOCK:
            tag = f'{MODEL_NAME}_BS{batch}_FT{ft*10}'
            print(tag)
            model = model_constructor(ft)
            model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='adam')
            history = model.fit(train_ds, epochs=100, verbose=0, validation_data=val_ds, callbacks=[EarlyStopping(patience=3)])
            save_history(history, tag)
            # time_converter(sum(cb_time.logs))
            epochs[ft] = len(history.history['loss'])
            loss[ft], accuracy[ft] = model.evaluate(test_ds)
        # PRINT THE RESULT
        # print('Count Epoch:', epochs)
        print(f' * * * * * {MODEL_NAME}_BS{batch}_LOSS * * * * * ')
        for item in loss:
            print(item)
        print(f' * * * * * {MODEL_NAME}_BS{batch}_ACCURACY * * * * * ')
        for item in accuracy:
            print(item)

 - - - - - TRIAL: 0  - - - - - 
Batch size: 32
Found 9980 files belonging to 2 classes.
Using 8982 files for training.
Found 9980 files belonging to 2 classes.
Using 998 files for validation.
Found 1000 files belonging to 2 classes.
resnet50_BS32_FT0
1000/1000 [==============================] - 24s 23ms/step - loss: 0.6793 - accuracy: 0.6940
resnet50_BS32_FT10
1000/1000 [==============================] - 26s 25ms/step - loss: 1.0158 - accuracy: 0.6990
resnet50_BS32_FT20
1000/1000 [==============================] - 54s 54ms/step - loss: 1.7032 - accuracy: 0.6720
resnet50_BS32_FT30
1000/1000 [==============================] - 25s 24ms/step - loss: 0.5952 - accuracy: 0.8590
resnet50_BS32_FT40
1000/1000 [==============================] - 25s 25ms/step - loss: 0.5384 - accuracy: 0.8690
resnet50_BS32_FT50
1000/1000 [==============================] - 21s 20ms/step - loss: 0.4929 - accuracy: 0.8760
resnet50_BS32_FT60
1000/1000 [==============================] - 24s 24ms/step - loss: 0.5737 - a

1000/1000 [==============================] - 18s 18ms/step - loss: 0.4521 - accuracy: 0.8930
resnet50_BS32_FT50
1000/1000 [==============================] - 21s 21ms/step - loss: 0.5590 - accuracy: 0.8450
resnet50_BS32_FT60
1000/1000 [==============================] - 21s 21ms/step - loss: 0.5925 - accuracy: 0.8530
resnet50_BS32_FT70
1000/1000 [==============================] - 12s 11ms/step - loss: 0.4674 - accuracy: 0.8580
resnet50_BS32_FT80
1000/1000 [==============================] - 17s 17ms/step - loss: 0.5579 - accuracy: 0.8780
resnet50_BS32_FT90
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3453 - accuracy: 0.8770
resnet50_BS32_FT100
1000/1000 [==============================] - 11s 11ms/step - loss: 0.4276 - accuracy: 0.8690
 * * * * * resnet50_BS32_LOSS * * * * * 
0.6062365174293518
1.0236619710922241
0.7001902461051941
0.8035963773727417
0.4520939290523529
0.5590106844902039
0.5925477147102356
0.4673694372177124
0.5579063296318054
0.34533870220184326
0.